Action1	cnews 中文文本分类：
由清华大学根据新浪新闻RSS订阅频道2005-2011年间的历史数据筛选过滤生成
训练集 50000
验证集 5000
测试集 10000
词汇（字） 5000
10个分类，包括：'体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐'"	

1、完成代码（30points）
2、结果正确（30points）

In [ ]:
import numpy as np
import torch
from torch import optim
from torch import nn
from model import TextRNN
from cnews_loader import read_vocab, read_category, process_file

In [ ]:
# 设置数据
train_file = 'cnews.train.txt'
test_file = 'cnews.test.txt'
val_file = 'cnews.val.txt'
vocab_file = 'cnews.vocab.txt'

In [ ]:
def train(Train_Epoch):
    model = TextRNN().cuda()
    # 定义损失函数
    Loss = nn.MultiLabelSoftMarginLoss()
    # 定义优化器
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    best_val_acc = 0
    # 训练
    for epoch in range(Train_Epoch):
        print('epoch=', epoch)
        # 分批训练
        for step, (x_batch, y_batch) in enumerate(train_loader):
            x = x_batch.cuda()
            y = y_batch.cuda()
            out = model(x)
            loss = Loss(out, y)
            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # 计算准确率
            accuracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
        print('train loss=', loss)
        print('train accuracy:', accuracy)
        # 对模型进行验证
        if (epoch+1) % 5 == 0:
            for step, (x_batch, y_batch) in enumerate(val_loader):
                x = x_batch.cuda()
                y = y_batch.cuda()
                out = model(x)
                accuracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
                if accuracy > best_val_acc:
                    torch.save(model, "model.pkl")
                    best_val_acc = accuracy
                    print('model.pkl saved')
                    print('val accuracy:', accuracy)
    return model

In [ ]:
# 获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
categories

In [ ]:
# 获取训练文本中所有出现过的字及其所对应的id
words, word_to_id = read_vocab('cnews.vocab.txt')
words

In [ ]:
# 获取训练数据每个字的id和对应标签的one-hot形式
x_train, y_train = process_file(train_file, word_to_id, cat_to_id, 600)
x_train

In [ ]:
x_val, y_val = process_file(val_file, word_to_id, cat_to_id, 600)

In [ ]:
import torch.utils.data as Data#将数据分批次需要用到
# 设置GPU
cuda = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [ ]:
x_train, y_train = torch.LongTensor(x_train), torch.Tensor(y_train)
x_val, y_val = torch.LongTensor(x_val), torch.Tensor(y_val)

In [ ]:
# 训练参数设置
Batch_Size = 256 #设置批次大小
Train_Epoch = 40

In [ ]:
train_dataset = Data.TensorDataset(x_train, y_train)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=Batch_Size, shuffle=True)
val_dataset = Data.TensorDataset(x_val, y_val)
val_loader = Data.DataLoader(dataset=val_dataset, batch_size=Batch_Size)

In [ ]:
# 模型训练
model = train(Train_Epoch)